In [1]:
import numpy as np
import pandas as pd

In [182]:
data = pd.read_csv('/home/kishora/Documents/Datasets/allYearHarangi.csv')
data=data.drop(columns=['Dates'])

In [186]:
transformer = Normalizer().fit(data)
dat=transformer.transform(data)
dat=pd.DataFrame(dat)

In [187]:
xx=[]
y=[]
for i in range(2900):
    z=[]
    X=[]
    for j in range(i,i+3):
        a=[]
        s=[]
        s.append(dat.iloc[j,-1])
        s.append(dat.iloc[j,3])
        a.append(s)
        
        X.extend(a)
    z.append(dat.iloc[i+4,-1])
    z.append(dat.iloc[i+4,3])
    y.append(z)
    xx.append(X)

In [189]:
xx=np.array(xx)
y=np.array(y)
y.shape


(2900, 2)

In [26]:
data.iloc[18+4,3]

100.0

In [153]:
xx=np.array(xx)
y=np.array(y)
y.shape

(2900, 2)

In [38]:

xx[176]

array([[  41.16666667, 5117.        ],
       [  24.25      , 4004.        ],
       [  24.        , 6761.        ]])

In [103]:
out = bias+np.dot(w[0],xx[:,0].T)+np.dot(w[1],xx[:,1].T)+np.dot(w[2],xx[:,2].T)

In [60]:
(bias.T+xx[:,0]).shape

(2400, 2)

In [65]:
np.dot(w[0],xx[:,0].T).shape

(2, 2400)

In [50]:
from sklearn import metrics

In [70]:
xx[967,0].T

array([   5.825, 4942.   ])

In [75]:
out[0]

array([ 353.47083255,  332.62034435,  272.40974041, ..., 6306.40610643,
       7462.13850505, 7388.89777237])

In [211]:
xx[:,0].T.shape

(2, 2900)

In [174]:
import math

In [245]:
math.sqrt(metrics.mean_squared_error(y.T,out))

3.40713756343876e+19

In [81]:
def sigmoid (x):
    return (1/(1 + np.exp(-x)))
def der(x):
    return (x*(1-x))

In [242]:
out=np.zeros((2,2400),dtype=np.float64)

In [243]:
w=[]
w.append(np.random.uniform(size=(2,2)))
w.append(np.random.uniform(size=(2,2)))
w.append(np.random.uniform(size=(2,2)))
bias= np.random.uniform(size=(2,1))

In [247]:
x=0
for i in range(5):
    l=np.dot(w[0],xx[:,0].T)
    m=np.dot(w[1],xx[:,1].T)
    n = np.dot(w[2],xx[:,2].T)
    out = bias+l+m+n
    x=(y.T-out)
    err = der(out)*x
    for i in range(len(w)):
        d=err.T*0.2
        w[i]+=xx[:,i].T.dot(d)
# # who += a.T.dot(err)*r
# # wih +=inp.T.dot(errh)*r

In [214]:
err.shape

(2, 2900)

In [206]:
 xx[:,0].shape

(2900, 2)

In [140]:
    out = bias+np.dot(w[0],xx[:,0].T)+np.dot(w[1],xx[:,1].T)+np.dot(w[2],xx[:,2].T)
    x=(y.T-out)
    err = der(out)*x
    for i in range(len(w)):
        w[i]-=xx[:,i].T.dot(err.T)*0.2

/home/kishora/PycharmProjects/abc/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in multiply
  after removing the cwd from sys.path.


In [138]:
pd.DataFrame(out).isna()

,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [122]:
out.shape

(2, 2400)

In [123]:
out

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [147]:
inp=[]
y=[]
for i in range(2400,2900):
    z=[]
    X=[]
    for j in range(i,i+3):
        a=[]
        s=[]
        s.append(data.iloc[j,-1])
        s.append(data.iloc[j,3])
        a.append(s)
        
        X.extend(a)
    z.append(data.iloc[i+4,-1])
    z.append(data.iloc[i+4,3])
    y.append(z)
    inp.append(X)


In [149]:
inp = np.array(inp)
y=np.array(y)

In [150]:
m = bias+np.dot(w[0],inp[:,0].T)+np.dot(w[1],inp[:,1].T)+np.dot(w[2],inp[:,2].T)

In [151]:
math.sqrt(metrics.mean_squared_error(y.T,m))

/home/kishora/PycharmProjects/abc/venv/lib/python3.6/site-packages/sklearn/metrics/regression.py:243: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,


inf